In [3]:
import datetime as dt
from utils import get_day_df
date = dt.datetime.now() - dt.timedelta(days=12)
s="badboll"
last_day = get_day_df(s,date.strftime("%y%m%d")) 
last_day


,Datetime,WR1_P,WR1_S1_P,WR1_sum,WR1_S1_Udc,WR2_P,WR2_S1_P,WR2_sum,WR2_S1_Udc,WR3_P,WR3_S1_P,WR3_sum,WR3_S1_Udc
0,2025-11-05 23:55:00,0,0,0,0,0,0,33375,0,0,0,34348,0
1,2025-11-05 23:50:00,0,0,0,0,0,0,33375,0,0,0,34348,0
2,2025-11-05 23:45:00,0,0,0,0,0,0,33375,0,0,0,34348,0
3,2025-11-05 23:40:00,0,0,0,0,0,0,33375,0,0,0,34348,0
4,2025-11-05 23:35:00,0,0,0,0,0,0,33375,0,0,0,34348,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
257,2025-11-05 00:20:00,0,0,0,0,0,0,0,0,0,0,0,0
258,2025-11-05 00:15:00,0,0,0,0,0,0,0,0,0,0,0,0
259,2025-11-05 00:10:00,0,0,0,0,0,0,0,0,0,0,0,0
260,2025-11-05 00:05:00,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
import pandas as pd
import requests
import re
from io import StringIO
from datetime import datetime
from typing import List, Tuple

def get_day_df_long(standort: str, date_str: str) -> pd.DataFrame:
    """
    Lädt Loggerdaten (min<date>.js) und liefert ein "long" DataFrame mit diesen Spalten:
      - Datetime (pd.Timestamp)
      - wr (int)    (z.B. 1, 2)
      - string (int) (-1 wenn kein String, sonst 1,2,...)
      - sensor (str) ('P','sum','Udc','T',...)
      - value (float)
    """
    def make_column_metadata(n_cols: int, wr_label: str) -> List[Tuple[str,int]]:
        """
        Baut eine Liste (sensor, string) für jede Spalte nach der Logik:
          [WR_P] + [WR_S1_P ... WR_Sn_P] + [WR_sum] + [WR_S1_Udc ...] (+ [WR_T] optional)
        Rückgabe: Liste mit Länge n_cols, Eintrag z.B. ('P', -1) oder ('P', 1) oder ('Udc', 1) ...
        """
        if n_cols < 2:
            raise ValueError(f"{wr_label}: Ungültige Anzahl an Spalten ({n_cols})")

        # Anzahl Strings (Anzahl Sx-Paare)
        n_strings = (n_cols - 2) // 2

        meta: List[Tuple[str,int]] = []
        # erste Spalte = P (kein string)
        meta.append(("P", -1))
        # danach S1_P ... S{n}_P
        for i in range(1, n_strings + 1):
            meta.append(("P", i))
        # dann sum (kein string)
        meta.append(("sum", -1))
        # danach S1_Udc ... S{n}_Udc
        for i in range(1, n_strings + 1):
            meta.append(("Udc", i))
        # optionales T am Ende
        if (n_cols - 2) % 2 == 1:
            meta.append(("T", -1))

        # Sanity check
        if len(meta) != n_cols:
            raise RuntimeError("Interner Fehler bei meta-Erstellung")

        return meta

    try:
        # URL zusammenbauen
        base_url = f"https://www.oekumenische-energiegenossenschaft.de/datenlogger/{standort}/visualisierung/"
        heute = datetime.now().strftime("%y%m%d")
        file_name = "min_day.js" if date_str == heute else f"min{date_str}.js"
        url = base_url + file_name

        # download
        response = requests.get(url)
        if response.status_code != 200:
            return pd.DataFrame()  # oder: f"Download fehlgeschlagen: {response.status_code}"

        matches = re.findall(r'="([^"]+)"', response.text)
        if not matches:
            return pd.DataFrame()

        raw_data = "\n".join(matches).replace("|", ",")
        raw_df = pd.read_csv(StringIO(raw_data), sep=",", header=None)

        # Zeitspalte parsen + Spaltennamen setzen
        raw_df[0] = pd.to_datetime(raw_df[0], format="%d.%m.%y %H:%M:%S")
        raw_df.columns = ["Datetime"] + [f"WR{i+1}" for i in range(len(raw_df.columns) - 1)]

        # Ergebnis sammeln
        long_parts: List[pd.DataFrame] = []

        # für jede Wechselrichter-Spalte
        for wr_col in raw_df.columns[1:]:
            col_series = raw_df[wr_col].astype(str)  # sicherstellen, str für split
            split_df = col_series.str.split(";", expand=True)

            # konvertiere zu float — falls leere Werte vorhanden sind, bleiben NaN
            split_df = split_df.apply(pd.to_numeric, errors="coerce")

            # Index beibehalten, damit Datetime sauber zugeordnet werden kann
            split_df.index = raw_df.index

            # Metadaten generieren (sensor, string) pro resultierender Spalte
            meta = make_column_metadata(split_df.shape[1], wr_col)

            # melt: originaler index als id mitnehmen
            melted = split_df.reset_index().melt(id_vars="index", var_name="col_idx", value_name="value")

            # wr index extrahieren (z.B. aus "WR2" -> 2)
            wr_idx_match = re.search(r'(\d+)', wr_col)
            wr_idx = int(wr_idx_match.group(1)) if wr_idx_match else None

            # mappe col_idx (als int) auf meta
            def map_meta(col_idx: int) -> Tuple[int, str]:
                # col_idx ist 0-basierter Spaltenindex nach dem split
                col_idx = int(col_idx)
                sensor, stringnum = meta[col_idx]
                return stringnum, sensor

            mapped = melted["col_idx"].apply(map_meta)
            melted["string"] = mapped.apply(lambda x: x[0])
            melted["sensor"] = mapped.apply(lambda x: x[1])
            melted["wr"] = wr_idx

            # Datetime korrekt zuordnen über den originalen row-index
            melted["Datetime"] = melted["index"].map(raw_df["Datetime"])

            # Spaltenreihenfolge wie gewünscht
            part = melted[["Datetime", "wr", "string", "sensor", "value"]]
            long_parts.append(part)

        # alles zusammenfügen und Typen setzen
        result = pd.concat(long_parts, ignore_index=True)
        result["wr"] = result["wr"].astype(int)
        result["string"] = result["string"].astype(int)

        return result

    except Exception as e:
        # Bei Fehlern leere DF zurückgeben oder Fehlertext — hier leeres DF
        return pd.DataFrame()


date = datetime.now()
s = "badboll"
last_day = get_day_df_long(s, date.strftime("%y%m%d"))
# sortieren und Beispiel-Filter:
last_day = last_day.sort_values(["Datetime", "wr", "string"]).reset_index(drop=True)
# Beispiel: Zeilen für 2025-11-17 11:20
ts = pd.Timestamp("2025-11-17 12:20:00")
last_day.loc[last_day.Datetime == ts]#wr1 2018 4000


,Datetime,wr,string,sensor,value
1524,2025-11-17 12:20:00,1,-1,P,0
1525,2025-11-17 12:20:00,1,-1,sum,0
1526,2025-11-17 12:20:00,1,1,P,0
1527,2025-11-17 12:20:00,1,1,Udc,0
1528,2025-11-17 12:20:00,2,-1,P,1982
1529,2025-11-17 12:20:00,2,-1,sum,1532
1530,2025-11-17 12:20:00,2,1,P,2045
1531,2025-11-17 12:20:00,2,1,Udc,477
1532,2025-11-17 12:20:00,3,-1,P,2018
1533,2025-11-17 12:20:00,3,-1,sum,1571


In [65]:
2018+1982


4000

In [ ]:
from tqdm import tqdm
import datetime as dt
import pandas as pd
from tqdm import tqdm

def update_all_days():
    standorte = ["badboll","esslingen","geislingen","holzgerlingen","hospitalhof",
                 "karlsruhe","mettingen","muensingen","tuebingen","waiblingen"]
    
    end = dt.datetime.now()
    days_back = 30
    start = end - dt.timedelta(days=days_back)
    required_dates = [start + dt.timedelta(days=i) for i in range((end - start).days + 1)]

    new_data = []
    for s in tqdm(standorte):
        missing_dates = required_dates
        for date in missing_dates:
            temp = get_day_df_long(s, date.strftime("%y%m%d"))
            if isinstance(temp, pd.DataFrame) and temp.shape[0] > 0:
                # Standort-Spalte hinzufügen
                temp["standort"] = s
                new_data.append(temp)

    if new_data:
        return pd.concat(new_data, ignore_index=True).drop_duplicates()
    else:
        return pd.DataFrame()

# Beispielaufruf
res = update_all_days()
res.to_parquet("detaildaten.parquet",index=False)

100%|██████████| 10/10 [00:31<00:00,  3.15s/it]


,Datetime,wr,string,sensor,value,standort
0,2025-10-18 23:55:00,1,-1,P,0,badboll
1,2025-10-18 23:50:00,1,-1,P,0,badboll
2,2025-10-18 23:45:00,1,-1,P,0,badboll
3,2025-10-18 23:40:00,1,-1,P,0,badboll
4,2025-10-18 23:35:00,1,-1,P,0,badboll
...,...,...,...,...,...,...
592665,2025-07-22 06:10:00,2,-1,T,16,tuebingen
592666,2025-07-22 06:06:00,2,-1,T,16,tuebingen
592667,2025-07-22 06:00:00,2,-1,T,15,tuebingen
592668,2025-07-22 05:55:00,2,-1,T,15,tuebingen


In [ ]:
res.to_parquet("detaildaten.parquet",index=False)


In [70]:
import polars as pl
df_polars = pl.scan_parquet("detaildaten.parquet")
heute = date.today().date()
standort="badboll"
df_filtered = (
        df_polars
        .filter(
            (pl.col("standort").str.to_lowercase() == standort.lower()) &
            (pl.col("Datetime").dt.date() == heute) &
            (pl.col("string") == -1) &
            (pl.col("sensor") == "P")
        )
        .group_by("Datetime")
        .agg(pl.col("value").sum().alias("P_gesamt"))
        .sort("Datetime")
        .collect(engine="streaming")
    )

df_filtered.to_pandas()

,Datetime,P_gesamt
0,2025-11-17 00:40:00,0
1,2025-11-17 00:45:00,0
2,2025-11-17 00:50:00,0
3,2025-11-17 00:55:00,0
4,2025-11-17 01:00:00,0
...,...,...
179,2025-11-17 16:40:00,0
180,2025-11-17 16:45:00,0
181,2025-11-17 16:50:00,0
182,2025-11-17 16:55:00,0


,Datetime,wr,string,sensor,value,standort
0,2025-11-17 17:00:00,1,-1,P,0,muensingen
1,2025-11-17 16:55:00,1,-1,P,0,muensingen
2,2025-11-17 16:50:00,1,-1,P,0,muensingen
3,2025-11-17 16:45:00,1,-1,P,0,muensingen
4,2025-11-17 16:40:00,1,-1,P,0,muensingen
...,...,...,...,...,...,...
8395,2025-11-17 00:20:00,6,-1,T,0,muensingen
8396,2025-11-17 00:15:00,6,-1,T,0,muensingen
8397,2025-11-17 00:10:00,6,-1,T,0,muensingen
8398,2025-11-17 00:05:00,6,-1,T,0,muensingen


In [42]:
df_filtered

Datetime,P_sum
datetime[ns],f64
2025-11-16 00:00:00,0.0
2025-11-16 00:05:00,0.0
2025-11-16 00:10:00,0.0
2025-11-16 00:15:00,0.0
2025-11-16 00:20:00,0.0
…,…
2025-11-16 16:30:00,119.0
2025-11-16 16:35:00,70.0
2025-11-16 16:40:00,38.0


In [ ]:
import polars as pl
from datetime import date
import numpy as np

def get_today_p_values(standort: str, file_path="detaildaten.parquet") -> np.ndarray:
    """
    Liest die Parquet-Datei im Long-Format und gibt die P-Werte des heutigen Tages
    für einen Standort als NumPy-Array zurück.
    """
    heute = date.today()

    # Datei laden als LazyFrame
    df_polars = pl.scan_parquet(file_path)

    # Filter, Gruppierung, Summierung
    df_filtered = (
        df_polars
        .filter(
            (pl.col("standort").str.to_lowercase() == standort.lower()) &
            (pl.col("Datetime").dt.date() == pl.lit(heute)) &
            (pl.col("sensor") == "P") &
            (pl.col("sensor") == "P")
        )
        .group_by("Datetime")
        .agg(pl.col("value").sum().alias("P_sum"))
        .sort("Datetime")
        .collect(engine="streaming")
    )

    # NumPy-Array zurückgeben
    return np.trim_zeros(df_filtered["P_sum"].to_numpy(), 'fb')

get_today_p_values("geislingen")

array([], dtype=float64)

In [2]:
import sys
print(sys.version)

3.12.9 (main, Feb  5 2025, 18:58:23) [Clang 19.1.6 ]


In [1]:
from update_ertragsdaten import update_ertrag
from update_leistungsdaten import update_leistung  
update_ertrag()
update_leistung()

Ertragsdaten aktualisiert!


100%|██████████| 10/10 [00:23<00:00,  2.31s/it]
